<a href="https://colab.research.google.com/github/GalalMohammed/SHAI_Training_2022-b_Tasks/blob/main/Task_6_exercises_answered.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](logo1.jpg)

# **shAI Training 2021 | Level 1**

## Task #6 (End-to-End ML Project {part_2})

## Welcome to the exercises for reviewing second part of end to end ML project.
**Make sure that you read and understand ch2 from the hands-on ML book (page 72 to the end of the chapter ) before start with this notebook.**

**If you stuck with anything reread that part from the book and feel free to ask about anything in the messenger group as you go along.**

 ## Good Luck : )

## first run the following cell for the first part of the project to continue your work 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [2]:
import os
import tarfile
import urllib
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
def load_housing_data(housing_path=HOUSING_PATH):
   csv_path = os.path.join(housing_path, "housing.csv")
   return pd.read_csv(csv_path)
   
fetch_housing_data()
housing = load_housing_data()

rooms_ix, bedrooms_ix, population_ix, household_ix = [
    list(housing.columns).index(col)
    for col in ("total_rooms", "total_bedrooms", "population", "households")]

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
housing = train_set.drop("median_house_value", axis=1)
housing_labels = train_set["median_house_value"].copy()

housing_num = housing.drop("ocean_proximity", axis=1)
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
 ('imputer', SimpleImputer(strategy="median")),
 ('attribs_adder', CombinedAttributesAdder()),
 ('std_scaler', StandardScaler())])

full_pipeline = ColumnTransformer([
 ("num", num_pipeline, num_attribs),
 ("cat", OneHotEncoder(), cat_attribs)])

housing_prepared = full_pipeline.fit_transform(housing)

# 1- Select and Train a Model

# Let’s first train a LinearRegression model 

In [3]:
# CODE HERE
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared,housing_labels)

LinearRegression()

# First try it out on a few instances from the training set:


In [4]:
housing_preparedDf = pd.DataFrame(housing_prepared)
some_data = housing_preparedDf.iloc[:5]
some_labels = housing_labels.iloc[:5]

In [5]:
# CODE HERE
housing_predictions = lin_reg.predict(some_data)
housing_predictions

array([181746.54359616, 290558.74973505, 244957.50017771, 146498.51061398,
       163230.42393939])

# measure this regression model’s RMSE on the whole training set 
* using Scikit-Learn’s mean_squared_error() function:

In [6]:
from sklearn.metrics import mean_squared_error

In [7]:
# CODE HERE
lin_mse = mean_squared_error(housing_labels, lin_reg.predict(housing_prepared))
lin_rmse = np.sqrt(lin_mse)
lin_rmse

67593.20745775253

# judge on the RMSE result for this model 
write down your answar 

In [8]:
lin_rmse / train_set["median_house_value"].mean()

0.32623039827101735

your answer goes here:
The error rate is about 32.62%, which seems bad.

# Let’s train a Decision Tree Regressor model 
## more powerful model

In [9]:
from sklearn.tree import DecisionTreeRegressor 

In [10]:
# CODE HERE
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared,housing_labels)

DecisionTreeRegressor()

# Now evaluate the model on the training set 
* using Scikit-Learn’s mean_squared_error() function:

In [11]:
# CODE HERE
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels,housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

# Explaine this result 
write down your answar

your answer goes here: The error rate is 0.0%, which means that the model overfits the data.

# Evaluation Using Cross-Validation

1-split the training set into 10 distinct subsets then train and evaluate the Decision Tree model

In [12]:
from sklearn.model_selection import cross_val_score

In [13]:
# CODE HERE
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring ="neg_mean_squared_error",cv = 10)
tree_rmse_scores = np.sqrt(-scores)

2- display the resultant scores and calculate its Mean and Standard deviation

In [14]:
# CODE HERE
print("Scores: ", tree_rmse_scores)
print("Mean: ", tree_rmse_scores.mean())
print("Standard Deviation: ", tree_rmse_scores.std())

Scores:  [65404.12249347 71031.82245825 68195.29345183 71305.21805524
 73326.52156145 67692.22052771 65666.0503553  68223.34825884
 65967.94619399 70752.8027695 ]
Mean:  68756.53461255884
Standard Deviation:  2585.355265093721


3-repaet the same steps to compute the same scores for the Linear Regression  model 

*notice the difference between the results of the two models*

In [15]:
# CODE HERE
scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring ="neg_mean_squared_error",cv = 10)
lin_rmse_scores = np.sqrt(-scores)
print("Scores: ", lin_rmse_scores)
print("Mean: ", lin_rmse_scores.mean())
print("Standard Deviation: ", lin_rmse_scores.std())

Scores:  [65000.67382615 70960.56056304 67122.63935124 66089.63153865
 68402.54686442 65266.34735288 65218.78174481 68525.46981754
 72739.87555996 68957.34111906]
Mean:  67828.38677377408
Standard Deviation:  2468.091395065227


## Let’s train one last model the RandomForestRegressor.

In [16]:
# CODE HERE
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

RandomForestRegressor()

# repeat the same steps to compute the same scores its Mean and Standard deviation for the Random Forest model

In [17]:
# CODE HERE
scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring ="neg_mean_squared_error",cv = 10)
forest_rmse_scores = np.sqrt(-scores)
print("Scores: ", forest_rmse_scores)
print("Mean: ", forest_rmse_scores.mean())
print("Standard Deviation: ", forest_rmse_scores.std())

Scores:  [46990.20760464 51848.73897905 49833.24150305 52148.06771604
 52565.97924465 46889.51374175 47565.98828303 50434.14991735
 49427.06699377 50025.36517094]
Mean:  49772.83191542605
Standard Deviation:  1982.5187276705703


# Save every model you experiment with 
*using the joblib library*

In [18]:
# CODE HERE
import joblib
lin_filename = 'finalized_lin_model.sav'
tree_filename = 'finalized_tree_model.sav'
forest_filename = 'finalized_forest_model.sav'
joblib.dump(lin_reg, lin_filename)
joblib.dump(tree_reg, tree_filename)
joblib.dump(forest_reg, forest_filename)

['finalized_forest_model.sav']

## now you have a shortlist of promising models. You now need to
## fine-tune them!
# Fine-Tune Your Model

## 1- Grid Search
## evaluate all the possible combinations of hyperparameter values for the RandomForestRegressor 
*It may take a long time*

In [19]:
from sklearn.model_selection import GridSearchCV

In [20]:
# CODE HERE
param_grid = [{'n_estimators': [3,10,30], 'max_features':[2,4,6,8]},{'bootstrap':[False], 'max_features':[2,3,4],'n_estimators':[3,10]}]
grid_search = GridSearchCV(forest_reg,param_grid,cv = 10, scoring = 'neg_mean_squared_error',return_train_score = True)
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

with the evaluation scores

In [21]:
# CODE HERE
print("Best Params: ", grid_search.best_params_)
print("Best Estimator: ", grid_search.best_estimator_)
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"],cvres["params"]):
  print(np.sqrt(-mean_score), params)

Best Params:  {'max_features': 6, 'n_estimators': 30}
Best Estimator:  RandomForestRegressor(max_features=6, n_estimators=30)
63157.6897743161 {'max_features': 2, 'n_estimators': 3}
54348.53986196158 {'max_features': 2, 'n_estimators': 10}
52089.44795310226 {'max_features': 2, 'n_estimators': 30}
59416.01384046177 {'max_features': 4, 'n_estimators': 3}
52096.84138862246 {'max_features': 4, 'n_estimators': 10}
49697.67224663595 {'max_features': 4, 'n_estimators': 30}
59592.42483551676 {'max_features': 6, 'n_estimators': 3}
51600.02831360728 {'max_features': 6, 'n_estimators': 10}
49428.40661750073 {'max_features': 6, 'n_estimators': 30}
58673.897025990875 {'max_features': 8, 'n_estimators': 3}
51256.27455872466 {'max_features': 8, 'n_estimators': 10}
49619.9616856113 {'max_features': 8, 'n_estimators': 30}
60919.41055599784 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
53702.21664496123 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
59135.25911964856 {'bootstra

# Analyze the Best Models and Their Errors
1-indicate the relative importance of each attribute

In [22]:
# CODE HERE
grid_search.best_estimator_.feature_importances_

array([6.98931519e-02, 6.13244386e-02, 4.27481480e-02, 1.82547591e-02,
       1.73161392e-02, 1.77330276e-02, 1.62845755e-02, 3.45830167e-01,
       6.82892412e-02, 1.05805645e-01, 5.76769216e-02, 6.38979232e-03,
       1.63765535e-01, 1.18115710e-04, 3.85724787e-03, 4.71309412e-03])

2-display these importance scores next to their corresponding attribute names:

In [23]:
housing

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
14196,-117.03,32.71,33.0,3126.0,627.0,2300.0,623.0,3.2596,NEAR OCEAN
8267,-118.16,33.77,49.0,3382.0,787.0,1314.0,756.0,3.8125,NEAR OCEAN
17445,-120.48,34.66,4.0,1897.0,331.0,915.0,336.0,4.1563,NEAR OCEAN
14265,-117.11,32.69,36.0,1421.0,367.0,1418.0,355.0,1.9425,NEAR OCEAN
2271,-119.80,36.78,43.0,2382.0,431.0,874.0,380.0,3.5542,INLAND
...,...,...,...,...,...,...,...,...,...
11284,-117.96,33.78,35.0,1330.0,201.0,658.0,217.0,6.3700,<1H OCEAN
11964,-117.43,34.02,33.0,3084.0,570.0,1753.0,449.0,3.0500,INLAND
5390,-118.38,34.03,36.0,2101.0,569.0,1756.0,527.0,2.9344,<1H OCEAN
860,-121.96,37.58,15.0,3575.0,597.0,1777.0,559.0,5.7192,<1H OCEAN


In [24]:
# CODE HERE
extra_attribs = ["population_per_household","bedroom_per_room","rooms_per_household"]
num_attribs = list(housing_num.columns)
attributes = num_attribs + extra_attribs + cat_attribs
sorted(zip(grid_search.best_estimator_.feature_importances_,attributes),reverse = True)

[(0.3458301666807556, 'median_income'),
 (0.10580564540549293, 'bedroom_per_room'),
 (0.06989315186323877, 'longitude'),
 (0.06828924117043571, 'population_per_household'),
 (0.06132443862102749, 'latitude'),
 (0.05767692156853374, 'rooms_per_household'),
 (0.04274814804794142, 'housing_median_age'),
 (0.018254759051086842, 'total_rooms'),
 (0.017733027579004662, 'population'),
 (0.017316139215679666, 'total_bedrooms'),
 (0.01628457554622594, 'households'),
 (0.006389792322480908, 'ocean_proximity')]

## Now is the time to evaluate the final model on the test set.
# Evaluate Your System on the Test Set

1-get the predictors and the labels from your test set

In [25]:
# CODE HERE
Y_test = test_set.drop("median_house_value", axis=1)
Y_test_labels = test_set["median_house_value"].copy()

2-run your full_pipeline to transform the data

In [30]:
# CODE HERE
Y_test_prepared = full_pipeline.transform(Y_test)

3-evaluate the final model on the test set

In [31]:
# CODE HERE
final_model = grid_search.best_estimator_
final_predictions = final_model.predict(Y_test_prepared)
final_mse = mean_squared_error(Y_test_labels, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

66980.08562318474

# compute a 95% confidence interval for the generalization error 
*using scipy.stats.t.interval():*

In [32]:
from scipy import stats

In [33]:
# CODE HERE
squared_errors = (final_predictions - Y_test_labels)
stats.t.interval(.95, len(squared_errors) - 1, loc=squared_errors.mean(), scale=stats.sem(squared_errors))

(19329.254458347732, 23205.91130392617)

# Great Job!
# #shAI_Club